In [1]:
# --- MODULE 3: THE SEMANTIC ENGINE ---
# Goal: Use Deep Learning to understand "Intent" beyond keywords.

# Install the Sentence-Transformers library, which provides pre-trained models for creating text embeddings.
!pip install -U sentence-transformers -q
# Import necessary classes from the library.
from sentence_transformers import SentenceTransformer, util
import torch

# 1. LOAD THE BRAIN
# This model has been 'Pre-trained' on billions of sentences to learn word relationships.
# 'all-MiniLM-L6-v2' is a small, efficient model that generates 384-dimensional embeddings.
model = SentenceTransformer('all-MiniLM-L6-v2')
print("🧠 Loaded the Sentence-Transformer model: 'all-MiniLM-L6-v2'")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🧠 Loaded the Sentence-Transformer model: 'all-MiniLM-L6-v2'


In [2]:
# 2. THE AGENT'S KNOWLEDGE BASE (Memory)
# This is a list of known intents or capabilities of our AI agent.
# Each string represents a distinct piece of information or an action the agent can take.
knowledge_base = [
    "I can help you book a flight to Paris.",             # Travel Intent
    "The restaurant is open until midnight.",            # Food Intent
    "Machine learning models require clean data.",       # Tech Intent
    "The weather today is sunny and bright."             # Weather Intent
]
print(f"📚 Agent's knowledge base loaded with {len(knowledge_base)} entries.")

📚 Agent's knowledge base loaded with 4 entries.


In [3]:
# 3. THE USER QUERY (Messy Input)
# This is the natural language input from the user.
query = "I need to reserve a plane ticket to France."

print(f"🌟 USER QUERY: '{query}'")
print("-" * 60)

# 4. ENCODING: The "Math Map"
# We convert text into 'Embeddings'—dense vectors of numbers. These vectors capture the semantic meaning.
# 'convert_to_tensor=True' ensures the output is a PyTorch tensor, which is efficient for calculations.
query_embedding = model.encode(query, convert_to_tensor=True)
kb_embeddings = model.encode(knowledge_base, convert_to_tensor=True)

# --- INSPECTION POINT: Let's see what a "Meaning Vector" looks like! ---
print(f"👀 Query Vector Shape: {query_embedding.shape} (384 different features of meaning)")
print(f"🔢 First 5 numbers of the Query Vector: {query_embedding[:5].tolist()}")
print(f"It's a mathematical representation of the meaning of the query.")
print("-" * 60)

🌟 USER QUERY: 'I need to reserve a plane ticket to France.'
------------------------------------------------------------
👀 Query Vector Shape: torch.Size([384]) (384 different features of meaning)
🔢 First 5 numbers of the Query Vector: [0.024892574176192284, 0.008249870501458645, -0.07432907074689865, -0.020831573754549026, 0.04562453553080559]
It's a mathematical representation of the meaning of the query.
------------------------------------------------------------


In [4]:
# 5. SEMANTIC SEARCH: Conceptual Matching
# Instead of counting words, we measure 'Cosine Similarity' between the query embedding and knowledge base embeddings.
# Cosine similarity tells us how similar two vectors are in direction, indicating semantic closeness.
# 'top_k=1' means we only want the single best match.
hits = util.semantic_search(query_embedding, kb_embeddings, top_k=1)[0]

# 6. RESULT & AGENTIC ACTION
# 'corpus_id' refers to the index of the best matching sentence in our 'knowledge_base' list.
best_match = knowledge_base[hits[0]['corpus_id']]
# 'score' is the cosine similarity score, ranging from -1 (opposite) to 1 (identical meaning).
confidence = hits[0]['score']

print(f"✅ AI INTENT MATCH: '{best_match}'")
print(f"This is the most semantically similar statement from the knowledge base.")
print(f"📊 CONFIDENCE SCORE: {confidence:.4f} (Scale of -1 to 1, higher is better)")
print(f"This score indicates how confident the AI is about the match.")

✅ AI INTENT MATCH: 'I can help you book a flight to Paris.'
This is the most semantically similar statement from the knowledge base.
📊 CONFIDENCE SCORE: 0.6388 (Scale of -1 to 1, higher is better)
This score indicates how confident the AI is about the match.


In [5]:
# 7. ADDED LOGIC: Handling Unknown Requests (Anti-Hallucination)
# We set a 'threshold' to decide if the match is good enough to act upon.
# This helps prevent the AI from giving irrelevant responses if it's not confident.
threshold = 0.5
print(f"Threshold for confident intent: {threshold}")

# If the confidence score is below the threshold, the intent is considered unclear.
if confidence < threshold:
    print("⚠️ STATUS: Intent Unclear. Agent will ask for clarification instead of guessing.")
    print("The AI is not confident enough to proceed with an action.")
else:
    print("🚀 STATUS: Intent Confirmed. Triggering 'Travel Booking Tool'...")
    print("The AI is confident and can now take a specific action based on the identified intent.")

Threshold for confident intent: 0.5
🚀 STATUS: Intent Confirmed. Triggering 'Travel Booking Tool'...
The AI is confident and can now take a specific action based on the identified intent.
